# Los arboles de regresion serian el siguiente punto de las regresiones 
* Lineal  cual sera el siguiente valor
* Logistica predecir si o no
* Arbol predecir en cual de las n categorias encajara el siguiente dato

## Arboles de Regresion

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("../datasets/boston/Boston.csv")
data.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [3]:
data.shape

(506, 14)

In [4]:
colnames = data.columns.values.tolist()
predictors = colnames[:13]
target =  colnames[13]
X = data[predictors]
Y = data[target]

In [5]:
from sklearn.tree import DecisionTreeRegressor

In [6]:
regtree = DecisionTreeRegressor (min_samples_split=30,min_samples_leaf=10,random_state=0)

In [7]:
regtree.fit(X,Y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=10,
           min_samples_split=30, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')

In [8]:
preds = regtree.predict(data[predictors])

In [9]:
data["preds"] = preds

In [10]:
data[["preds","medv"]][:15] ## si en la prediccion se repite el valor es por que caen en la misma categoria

,preds,medv
0,22.840000,24.0
1,22.840000,21.6
2,35.247826,34.7
3,35.247826,33.4
4,35.247826,36.2
5,24.058621,28.7
6,20.811111,22.9
7,20.020833,27.1
8,20.020833,16.5
9,20.020833,18.9


In [11]:
from  sklearn.tree import export_graphviz
with open("resources/boston_rtree.dot","w") as dotfile:
    export_graphviz(regtree,out_file=dotfile, feature_names=predictors)
    dotfile.close()
import os
from graphviz import Source
file = open("resources/boston_rtree.dot","r")
text = file.read()

In [12]:
#Source(text)
# alternativa http://www.webgraphviz.com/

In [13]:
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [14]:
cv = KFold(n=X.shape[0],n_folds = 10, shuffle=True, random_state=1)

In [15]:
tree = DecisionTreeRegressor(min_samples_split=30,min_samples_leaf=10,random_state=0 )
tree.fit(X,Y)
cross_score = cross_val_score(regtree,X,Y,scoring="neg_mean_squared_error", cv=cv, n_jobs=1)
print("Scores",cross_score)
score = np.mean(cross_score)
print("Score ",score, " margen 20 arriba o 20 abajo")


Scores [-13.64925886 -17.28987161 -16.98569707 -47.56954086  -9.26202865
 -17.23057023 -15.41541493 -31.33011027 -22.79877067  -9.54180723]
Score  -20.107307036443846  margen 20 arriba o 20 abajo


In [43]:
list(zip(predictors,regtree.feature_importances_))

[('crim', 0.03421203230639308),
 ('zn', 0.0),
 ('indus', 0.0011605887788380146),
 ('chas', 0.0),
 ('nox', 0.01856163073811432),
 ('rm', 0.6308568014337028),
 ('age', 0.01725115143448847),
 ('dis', 0.0013745115995791378),
 ('rad', 0.0),
 ('tax', 0.0023698305298803803),
 ('ptratio', 0.009333247332530954),
 ('black', 0.0),
 ('lstat', 0.28488020584647283)]

## Random Forest

In [17]:
from sklearn.ensemble import RandomForestRegressor

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [39]:
# n_jobs = numero de hilos en el procesador
# obb_score = True va a hacer un muestreo automatico
# n_estimators = Numero de arboles
forest = RandomForestRegressor(n_jobs=2, oob_score=True, n_estimators = 100)
forest.fit(X,Y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=2,
           oob_score=True, random_state=None, verbose=0, warm_start=False)

In [40]:
data["rforest_pred"] = forest.oob_prediction_
data[["rforest_pred","medv"]][:15]

,rforest_pred,medv
0,29.318750,24.0
1,22.433333,21.6
2,34.003704,34.7
3,35.691304,33.4
4,34.085714,36.2
5,24.022222,28.7
6,19.681081,22.9
7,17.671875,27.1
8,18.676000,16.5
9,18.931707,18.9


In [41]:
data["rforest_error2"] = (data["rforest_pred"]-data["medv"])**2
error = sum(data["rforest_error2"])/len(data)
print("Error del modelo",str(error)," puntos")

Error del modelo 10.038979779744569  puntos


In [42]:
forest.oob_score_

0.8810822961308429

### 100 arboles y 5000 generar 10 puntos de error entonces en algunos casos tampoco es necesario crear tantos arboles